In [3]:
import glob
import pandas as pd
cat = glob.glob("train/train/cat.*.jpg")
dog = glob.glob("train/train/dog.*.jpg")
data = pd.DataFrame({
    "path":cat + dog,
    "ans":[0] * len(cat) + [1] * len(dog)
})
data

,path,ans
0,train/train\cat.0.jpg,0
1,train/train\cat.1.jpg,0
2,train/train\cat.10.jpg,0
3,train/train\cat.100.jpg,0
4,train/train\cat.1000.jpg,0
...,...,...
24995,train/train\dog.9995.jpg,1
24996,train/train\dog.9996.jpg,1
24997,train/train\dog.9997.jpg,1
24998,train/train\dog.9998.jpg,1


In [5]:
from keras.applications.vgg16 import VGG16
vgg = VGG16(include_top=False, input_shape=(224, 224, 3))
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [13]:
from keras.layers import Flatten, Dense, Dropout
from keras.models import Model
# Dense(256, activation="relu") -> 函式(e.g. return print)
# tensor = func(tensor)
for l in vgg.layers:
    l.trainable = False
x = Flatten()(vgg.output)
x = Dense(2048, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(2, activation="softmax")(x)
# Model(inputs=tensor, outputs=tensor)
cnn = Model(inputs=vgg.input, outputs=x)
cnn.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [15]:
# trainable一定要在compile前就設定
cnn.compile(loss="categorical_crossentropy",
            optimizer="adam",
            metrics=["accuracy"])

In [33]:
from sklearn.model_selection import train_test_split
# !!! 進入訓練階段以後, list(X), dataframe(X) np.array(O)
import numpy as np
from keras.utils.np_utils import to_categorical
x = np.array(data["path"])
y = np.array(data["ans"])
y_cat = to_categorical(y)
x_train, x_test, y_train, y_test = train_test_split(x, 
                                                    y_cat, 
                                                    test_size=0.1)

In [24]:
# Pillow(PIL)
# 處理一張照片給你看
from keras.preprocessing.image import load_img
# 一定使用那個模型的preprocess
from keras.applications.vgg16 import preprocess_input
img = load_img(x_train[0], target_size=(224, 224)).convert("RGB")
img_np = np.array(img)
preprocess_input(img_np)

array([[[70.061   , 59.221   , 52.32    ],
        [73.061   , 62.221   , 55.32    ],
        [75.061   , 64.221   , 57.32    ],
        ...,
        [58.060997, 45.221   , 38.32    ],
        [55.060997, 42.221   , 35.32    ],
        [59.060997, 40.221   , 34.32    ]],

       [[71.061   , 60.221   , 53.32    ],
        [74.061   , 63.221   , 56.32    ],
        [76.061   , 65.221   , 58.32    ],
        ...,
        [58.060997, 45.221   , 38.32    ],
        [55.060997, 42.221   , 35.32    ],
        [57.060997, 38.221   , 32.32    ]],

       [[72.061   , 61.221   , 54.32    ],
        [75.061   , 64.221   , 57.32    ],
        [77.061   , 66.221   , 59.32    ],
        ...,
        [58.060997, 45.221   , 38.32    ],
        [55.060997, 42.221   , 35.32    ],
        [57.060997, 38.221   , 32.32    ]],

       ...,

       [[84.061   , 82.221   , 73.32    ],
        [85.061   , 83.221   , 74.32    ],
        [87.061   , 85.221   , 76.32    ],
        ...,
        [76.061   , 65.221

In [34]:
# x:圖片, y:答案, batch:取幾個
# 回傳 (原始圖片[batch], 處理圖片[batch], 答案[batch])
def get_data(x, y, batch):
    idx = np.random.randint(0, x.shape[0], batch)
    # 原本未處理的圖片
    oriimgs = []
    # Preprocess過後的圖片
    preimgs = []
    for p in x[idx]:
        img = load_img(p, target_size=(224, 224)).convert("RGB")
        img_np = np.array(img)
        oriimgs.append(img_np)
        img_pre = preprocess_input(img_np) 
        preimgs.append(img_pre)

    oriimgs = np.array(oriimgs)
    preimgs = np.array(preimgs)
    ans = y[idx]
    return (oriimgs, preimgs, ans)

In [37]:
# 訓練: train_on_batch
# 驗證: test_on_batch
for i in range(10):
    print("次數:", i + 1)
    batch = 10
    train = get_data(x_train, y_train, batch)
    train_result = cnn.train_on_batch(train[1], train[2])
    print("[Train]:", train_result)
    # 如果可以把驗證的圖片設多一點
    test_batch = 20
    test = get_data(x_test, y_test, test_batch)
    test_result = cnn.test_on_batch(test[1], test[2])
    print("[Validate]:", test_result)
    print("-" * 30)

次數: 1
[Train]: [11.282667, 0.3]
[Validate]: [7.2531433, 0.55]
------------------------------
次數: 2
[Train]: [4.8354287, 0.7]
[Validate]: [8.864953, 0.45]
------------------------------
次數: 3
[Train]: [11.282667, 0.3]
[Validate]: [8.864952, 0.45]
------------------------------
次數: 4
[Train]: [9.670857, 0.4]
[Validate]: [7.2531433, 0.55]
------------------------------
次數: 5
[Train]: [11.282667, 0.3]
[Validate]: [7.2531433, 0.55]
------------------------------
次數: 6
[Train]: [11.282667, 0.3]
[Validate]: [7.2531433, 0.55]
------------------------------
次數: 7
[Train]: [4.8354287, 0.7]
[Validate]: [7.2531433, 0.55]
------------------------------
次數: 8
[Train]: [3.223619, 0.8]
[Validate]: [8.059048, 0.5]
------------------------------
次數: 9
[Train]: [9.670857, 0.4]
[Validate]: [8.059048, 0.5]
------------------------------
次數: 10
[Train]: [12.894476, 0.2]
[Validate]: [9.670857, 0.4]
------------------------------


In [38]:
# 秀一下問題給你看
# 問題1: 怎麼全部同一個預測
# 問題2: 機率不該看到1 & 0
cnn.predict(test[1])

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)